In [1]:
import sys,os
import pickle
import anndata
import pandas as pd
import numpy as np
import scanpy as sc
import diffxpy.api as de
import statsmodels.stats.multitest as multi

In [2]:
adata = sc.read_h5ad(open("/nfs/team283/aa16/KR_NAS/data/CR_cells_AnnData.h5ad", "rb"))

In [ ]:
# Number of nuclei per AOI type:
print('CR cell ROIs mean nuclei:', np.mean(adata.obs['nuclei'].loc[adata.obs['Pool'] == 'CRcells']))
print('Residual ROIs mean nuclei:', np.mean(adata.obs['nuclei'].loc[adata.obs['Pool'] == 'Residual']))
# RELN expression per AOI type:
print('CR cell ROIs mean RELN expression:', np.mean(adata[adata.obs['Pool'] == 'CRcells',np.array(adata.var == 'RELN')].X))
print('Residual ROIs mean RELN expression:', np.mean(adata[adata.obs['Pool'] == 'Residual',np.array(adata.var == 'RELN')].X))
# NR2F1 expression in Frontal vs Occipital:
print('Frontal Cortex ROIs mean RELN expression:', np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (frontal lobe)', np.array(adata.var == 'NR2F1')].X))
print('Occipital Cortex ROIs mean RELN expression:', np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (occipital lobe)', np.array(adata.var == 'NR2F1')].X))

if np.mean(adata.obs['nuclei'].loc[adata.obs['Pool'] == 'CRcells']) > np.mean(adata.obs['nuclei'].loc[adata.obs['Pool'] == 'Residual']):
    raise ValueError()
elif np.mean(adata[adata.obs['Pool'] == 'CRcells',np.array(adata.var == 'RELN')].X) < np.mean(adata[adata.obs['Pool'] == 'Residual',np.array(adata.var == 'RELN')].X):
    raise ValueError()
elif np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (frontal lobe)', np.array(adata.var == 'NR2F1')].X) > np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (occipital lobe)', np.array(adata.var == 'NR2F1')].X):
    raise ValueError()
else:
    print('Input data looks sensible.')

In [3]:
adata.var_names = adata.var['SYMBOL']

In [4]:
adata = adata[adata.obs['Pool'] == 'CRcells',:]

In [5]:
adata.X = adata.layers['X_corrected_naive_cpm']

In [6]:
adata.obs['Tissue'].value_counts()

Foetal brain (temporal lobe)     20
Foetal brain (occipital lobe)    20
Foetal brain (frontal lobe)      18
Foetal brain (parietal lobe)     16
Name: Tissue, dtype: int64

In [7]:
res = {}
for a in np.unique(adata.obs['Tissue']):
    print(a)
    adata_subset = adata.copy()
    adata_subset.obs['Tissue'] = adata_subset.obs['Tissue'].astype(str)
    adata_subset.obs['Tissue'].loc[adata_subset.obs['Tissue'] != a] = 'AA'
    res[a] = de.test.rank_test(
        data=adata_subset,
        grouping="Tissue",
        is_logged=False
    ).summary()

Foetal brain (frontal lobe)


/nfs/users/nfs_a/aa16/.local/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Foetal brain (occipital lobe)
Foetal brain (parietal lobe)
Foetal brain (temporal lobe)


In [15]:
for a in np.unique(adata.obs['Tissue']):
    print(a)
    res[a]['log2_mean_' + a] = np.array(np.log2(np.mean(adata[adata.obs['Tissue'] == a,res[a].loc[:,'gene']].X, axis = 0)))
    res[a]['log2_mean_Rest'] = np.array(np.log2(np.mean(adata[adata.obs['Tissue'] != a,res[a].loc[:,'gene']].X, axis = 0)))
    res[a] = res[a].sort_values('pval')
#     res[a] = res[a].drop(['zero_mean', 'zero_variance', 'mean'], axis = 1)
    print(np.sum(res[a].loc[:,'qval'] < 0.05))
    print(np.sum([res[a].loc[:,'qval'].iloc[i] < 0.05
 and np.abs(res[a].loc[:,'log2fc'].iloc[i]) > 0.5
 for i in range(len(res[a].loc[:,'qval']))]))
    res[a].to_csv('/nfs/team283/aa16/KR_NAS/resultLists/CR_DE_' + a + '_markers.csv', index = False)

Foetal brain (frontal lobe)
0
0
Foetal brain (occipital lobe)
1
1
Foetal brain (parietal lobe)
0
0
Foetal brain (temporal lobe)
6
6
